## 2.1 Data

In [3]:
from datasets import load_dataset

dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

In [4]:
# 데이터셋의 구조 확인
data = dataset
data

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [5]:
# 문서를 모두 합치고, 한국어 글자의 집합 만들기
ko_text = ''.join(data['train']['document'])
ko_chars = sorted(list(set((ko_text))))
ko_vocab_size = len(ko_chars)
print('총 글자 수 : ', ko_vocab_size)

총 글자 수 :  2701


In [6]:
# 한국어 글자를 토큰화
character_to_ids = {char:i for i, char in enumerate(ko_chars)}
ids_to_character = {i: char for i, char in enumerate(ko_chars)}
token_encode = lambda s:[character_to_ids[c] for c in s]
token_decode = lambda l: ''.join([ids_to_character[i] for i in l])
print(token_encode('안녕하세요 함께 인공지능을 공부하게 되어 반가워요.'))
print(token_decode(token_encode('안녕하세요 함께 인공지능을 공부하게 되어 반가워요.')))

[1909, 1169, 2546, 1770, 2008, 0, 2551, 1061, 0, 2064, 977, 2157, 1209, 2055, 0, 977, 1658, 2546, 949, 0, 1283, 1942, 0, 1593, 908, 2024, 2008, 2]
안녕하세요 함께 인공지능을 공부하게 되어 반가워요.


In [7]:
# 토큰화한 글자를 텐서로 변환
import torch

tokenized_data = torch.tensor(token_encode(ko_text), dtype=torch.long)
print(tokenized_data.shape, tokenized_data.dtype)
print(tokenized_data[:100])

torch.Size([22162967]) torch.int64
tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987, 2555,    0, 2546, 1593,
        1028,    0, 2015, 1485,    0,  965, 2107, 2060,    0, 1617, 2465, 1542,
        2064,    0, 1808, 2273,    0, 2603, 1236, 1477,    0, 2037, 2555,    0,
        2263, 1430, 2055,    0, 1028, 2019, 2062, 1028, 1441,    0, 2562, 1841,
        1213, 1221,    2,    0, 2451, 2650,    0, 1808, 2273,    0, 2142, 1787,
        1028, 1950, 2060,    0, 1558, 1468, 1119,    0, 2555, 1787, 1477,    0,
        2037, 2555,    0, 1553, 1967, 1024, 2051,    0, 1015, 1541, 1477,    0,
           7,    3, 2117,    0, 2026,    0, 2062, 1740,    0, 2603, 1236, 2546,
         968,    0, 1558, 1468])


In [8]:
# train, test 데이터 분할 (9:1 비율로)

n = int(0.9 * len(tokenized_data))
train_dataset = tokenized_data[:n]
test_dataset = tokenized_data[n:]

실제 학습 과정은 순서대로 이뤄지는 것이 아니다.  
`block_size`에 설정된 크기만큼의 청크(chunk) 단위로 무작위 **샘플링**해 학습을 진행한다.  
모델이 다양한 문맥에서 언어를 이해하고 생성하는 능력을 키운다.  
무작위 샘플링을 통해 모델이 데이터의 특정 부분에 과적합되는 것을 방지하고, 전체 데이터셋에 대해 고르게 학습할 수 있다.  

`block_size`는 context length라고 불린다.  
즉 모델이 한 번에 처리할 수 있는 토큰의 최대 길이를 뜻한다.  
모델의 성능과 효율성에 큰 영향을 미친다.  
`block_size`가 커지면 모델이 긴 문맥을 이해하게 해주지만, 더 많은 계산 자원을 필요로 한다.  
`block_size`가 작아지면 계산 효율성은 높아지지만, 모델의 문맥 이해 능력은 제한된다.  
적절한 `block_size`를 선택하는 것이 중요하다.  

In [9]:
block_size = 8

x = train_dataset[:block_size]
y = train_dataset[1:block_size+1]

for time in range(block_size):
    context = x[:time+1]    
    target = y[time]

    print(f'입력 텐서 : {context}') # 타겟 글자를 학습하는데 참고하는 텐서들.
    print(f'타겟 글자 : {target}')

입력 텐서 : tensor([1928])
타겟 글자 : 2315
입력 텐서 : tensor([1928, 2315])
타겟 글자 : 0
입력 텐서 : tensor([1928, 2315,    0])
타겟 글자 : 2105
입력 텐서 : tensor([1928, 2315,    0, 2105])
타겟 글자 : 1658
입력 텐서 : tensor([1928, 2315,    0, 2105, 1658])
타겟 글자 : 908
입력 텐서 : tensor([1928, 2315,    0, 2105, 1658,  908])
타겟 글자 : 0
입력 텐서 : tensor([1928, 2315,    0, 2105, 1658,  908,    0])
타겟 글자 : 1987
입력 텐서 : tensor([1928, 2315,    0, 2105, 1658,  908,    0, 1987])
타겟 글자 : 2555


실제 인공지능 모델 훈련 시에는 위 예시처럼 단일 글자 텐서만 입력으로 주어지지 않는다.  
여러 개의 텐서가 함께 묶여 입력으로 제공된다. 이를 batch라고 한다.  

`batch_size`는 한 번에 모델이 처리할 데이터의 양을 설정한다. '4'라면, 모델이 4개의 예제를 병렬로 처리할 수 있다는 뜻이다.  
`block_size`는 모델이 한 번에 볼 수 있는 글자의 수를 설정한다. '8'이라면, 모델이 8개의 글자를 하나의 시퀀스로 보고 학습을 수행한다.  

In [10]:
torch.manual_seed(1234)

batch_size = 4
block_size = 8  

def batch_function(mode):
    dataset = train_dataset if mode == 'train' else test_dataset
    idx = torch.randint(len(dataset) - block_size, (batch_size,))
    x = torch.stack([dataset[index:index+block_size] for index in idx])
    y = torch.stack([dataset[index+1:index+block_size+1] for index in idx])
    return x,y  

example_x, example_y = batch_function('train')

print('inputs : ', example_x.shape)
print('')
print('example_x의 실제값')
print(example_x)
print('-----------------------------')
print('targets : ', example_y.shape)
print('')
print('example_y의 실제 값')
print('example_y')
print('-----------------------------')

for size in range(batch_size):
    for t in range(block_size):
        context = example_x[size, :t+1]
        target = example_y[size, t]
        print(f'input : {context}, target : {target}')
    print('-----------------------------')
    print('-----------------------------')

inputs :  torch.Size([4, 8])

example_x의 실제값
tensor([[1764, 2555,    0, 1236, 2248,    0, 2017, 1976],
        [   0, 1966, 2157,    0, 1951, 2062,    0, 2548],
        [   0, 1304, 1485, 1586,    0, 1907, 2450,    0],
        [   3,    2,    6,    5,    1,    0,    5,    3]])
-----------------------------
targets :  torch.Size([4, 8])

example_y의 실제 값
example_y
-----------------------------
input : tensor([1764]), target : 2555
input : tensor([1764, 2555]), target : 0
input : tensor([1764, 2555,    0]), target : 1236
input : tensor([1764, 2555,    0, 1236]), target : 2248
input : tensor([1764, 2555,    0, 1236, 2248]), target : 0
input : tensor([1764, 2555,    0, 1236, 2248,    0]), target : 2017
input : tensor([1764, 2555,    0, 1236, 2248,    0, 2017]), target : 1976
input : tensor([1764, 2555,    0, 1236, 2248,    0, 2017, 1976]), target : 2546
-----------------------------
-----------------------------
input : tensor([0]), target : 1966
input : tensor([   0, 1966]), target : 2157


### 2.3 언어 모델 만들기

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F 

class semiGPT(nn.Module):
    def __init__(self, vocab_length):
        super().__init__()
        self.embedding_token_table = nn.Embedding(vocab_length, vocab_length)

    def forward(self, inputs, targets):
        logits = self.embedding_token_table(inputs)

        return logits
    
model = semiGPT(ko_vocab_size)
output = model(example_x,example_y)
print(output.shape)

torch.Size([4, 8, 2701])
